In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers

In [3]:
data = pd.read_csv("scraped_dataset/dynamic_pricing.csv")

In [4]:
sales_scaler = MinMaxScaler(feature_range=(0,1))
sales_scaler.fit(np.array([30,500]).reshape(-1,1))

MinMaxScaler()

In [5]:
# Create time series data
scaled_data = pd.DataFrame({
    'id':[],
    'week':[],
    'score': [],
    'price': [],
    'sales':[]
})
for product_id, group in data.groupby('id'):
    init_price = group['price'].iloc[0]
    price_range = [(init_price - (0.15 * init_price)) ,(init_price + (0.15 * init_price ))]
    scaler = MinMaxScaler(feature_range=(0,1), )
    scaler.fit(np.array(price_range).reshape(-1,1))
    group['price'] = scaler.transform(group['price'].values.reshape(-1,1))
    group['sales'] = sales_scaler.transform(group['sales'].values.reshape(-1,1))
    scaled_data = pd.concat([scaled_data, group])

In [6]:
# Group the data by product_id and create a dictionary of input/output sequences for each group
grouped_data = {}
for product_id, group in scaled_data.groupby('id'):
    input_seqs = []
    output_seqs = []
    for i in range(len(group) - 1):
        input_seq = group.iloc[i:i+2][['score', 'sales', 'price']].values.tolist()
        output_seq = [group.iloc[i+1]['price']]
        input_seqs.append(input_seq)
        output_seqs.append(output_seq)
    if len(input_seq) < 2:
        continue
    grouped_data[product_id] = (input_seqs, output_seqs)


In [7]:
delete_ids = []
for id in grouped_data:
    s= grouped_data[id]
    if len(s[0]) == 0:
        delete_ids.append(int(id))
for id in delete_ids:
    del grouped_data[id]

In [8]:
model = Sequential()
model.add(layers.LSTM(50, activation='relu', input_shape=(2, 3)))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

In [9]:
for product_id, data in grouped_data.items():
    X = data[0]
    y = data[1]
    model.fit(X, y, epochs=100, verbose=0)

In [12]:
model.save('./dynamic_pricing')

INFO:tensorflow:Assets written to: ./dynamic_pricing/assets


In [10]:
# prediction
for product_id, data in grouped_data.items():
    X = data[0]
    y = data[1]
    new_week = np.array([[[1.0, -2.8239423392966145, 0.5], [1.0, -2.821594590787435, 0.5]], [[1.0, -2.821594590787435, 0.5],[1.0, -2.8208902662346813, 0.48408226510776187]]])
    
    prediction = model.predict(X)
    print(f"Product ID {product_id}: predicted price for next week is {prediction[-1][0]}")

Product ID 0.0: predicted price for next week is 0.4799378514289856
Product ID 1.0: predicted price for next week is 0.5315628051757812
Product ID 3.0: predicted price for next week is 0.39526665210723877
Product ID 5.0: predicted price for next week is 0.5183670520782471
Product ID 6.0: predicted price for next week is 0.41913673281669617
Product ID 8.0: predicted price for next week is 0.41792717576026917
Product ID 9.0: predicted price for next week is 0.5007126927375793
Product ID 10.0: predicted price for next week is 0.5628650188446045
Product ID 11.0: predicted price for next week is 0.3838080167770386
Product ID 12.0: predicted price for next week is 0.38161367177963257
Product ID 14.0: predicted price for next week is 0.5315628051757812
Product ID 15.0: predicted price for next week is 0.34842097759246826
Product ID 16.0: predicted price for next week is 0.49709922075271606
Product ID 17.0: predicted price for next week is 0.5076217651367188
Product ID 19.0: predicted price fo